<a href="https://www.kaggle.com/code/uzairajokhio/titanic-survival-notebook?scriptVersionId=215063050" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
data = train_data
data.head(10)

# EDA

In [ ]:
print(f"Shape of train data: {train_data.shape}")
print(f"Shape of train data: {test_data.shape}")
print(f"Shape of train data: {data.shape}")

In [ ]:
print(data.info())

In [ ]:
data.describe()

In [ ]:
print(data.isnull().sum())
sns.heatmap(data.isnull(),cbar=False)
plt.show()

## Analyising Features

In [ ]:
fig, axes=plt.subplots(1,2,figsize=(12,6))
data['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=axes[0],shadow=True)
axes[0].set_title('Survived')
axes[0].set_ylabel('')
sns.countplot(x='Survived',data=data,ax=axes[1])
axes[1].set_title('Survived')
plt.show()

### Sex

In [ ]:
data.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,6))
sns.countplot(x='Sex', hue='Survived', data=data, ax=axes[0])
axes[0].set_title("Sex: Dead v Survived ")

data[['Sex', 'Survived']].groupby("Sex").mean().plot.bar(ax=axes[1])
axes[1].set_title("Survival Ratio Among Male and Female")

plt.show()


As we can see there was a high percentage of Female Survior 

### Pclass

In [ ]:
print(pd.crosstab(data['Pclass'], data['Survived'], margins=True))

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,6))

data['Pclass'].value_counts().plot.bar(ax=axes[0])
axes[0].set_title('Total number Pclass')

sns.countplot(x='Pclass', hue='Survived', data=data,ax=axes[1])
axes[1].set_title("")

plt.show()

### Age

In [ ]:
print(f'The youngest Passenger: {data["Age"].min()}')
print(f'The Oldest Passenger: {data["Age"].max()}')
print(f'The average age Passenger: {round(data["Age"].mean(),2)} ')

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,6))

sns.violinplot(x='Pclass',y='Age', data=data, hue='Survived', split=True, ax=axes[0])
axes[0].set_title("Survival among Pclass and Age")
sns.violinplot(x='Sex',y='Age', data=data, hue='Survived', split=True, ax=axes[1])
axes[1].set_title("Survival among Sex and Age")
plt.show()

### Observations:
1. The Survial Rate of young increase among the Pclass the survial rate of young looks good,
2. The survial Rate from age 20-40 in Pclass 1 is higher

### Now dealing with null value of Age

we can take the mean of the age and replace it with nan, but there is a problem we cant assign an average of 32.22 to a 5 year old,

however we can see that in the Name columns there are title assigns MR and MSS 
thus we can assign the mean values of Mr and Mrs to the respective groups.

In [ ]:
data['Initial'] = 0

for i in data:
    data['Initial'] = data['Name'].str.extract("([A-Za-z]+)\.")

In [ ]:
titles = data['Initial'].unique()
print(titles)

In [ ]:
pd.crosstab(data['Initial'], data['Sex']).T

In [ ]:
data['Initial'].replace(['Mme', 'Ms','Mlle','Dr','Major','Lady','Countess','Sir','Capt','Col','Jonkheer','Rev', 'Col', 'Don', "Dona"],
                        ['Miss', 'Miss','Miss','Mr','Mr',"Mrs",'Mrs','Mr','Mr', 'Mr','Other','Other','Other','Mr', 'Mrs'],inplace=True)

In [ ]:
data.groupby('Initial')['Age'].mean()

In [ ]:
data.loc[(data['Age'].isnull()) & (data['Initial']=='Master'),"Age"] = 5.48
data.loc[(data['Age'].isnull()) & (data['Initial']=='Miss'),"Age"] = 21.83
data.loc[(data['Age'].isnull()) & (data['Initial']=='Mr'),"Age"] = 32.54
data.loc[(data['Age'].isnull()) & (data['Initial']=='Mrs'),"Age"] = 37.04
data.loc[(data['Age'].isnull()) & (data['Initial']=='Other'),"Age"] = 44

In [ ]:
data['Age'].isnull().any()

### Embarked

In [ ]:
pd.crosstab([data['Embarked'], data['Pclass']], [data['Sex'],data['Survived']], margins=True).T

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(16,8))

sns.countplot(x='Embarked', data=data, ax=axes[0,0])
axes[0,0].set_title("Total number of Embarked")
sns.countplot(x='Embarked', hue='Sex', data=data, ax=axes[0,1])
axes[0,1].set_title('Male vs Female')
sns.countplot(x='Embarked',hue='Pclass', data=data, ax=axes[1,0])
axes[1,0].set_title('Pclass')
sns.countplot(x='Embarked',hue='Survived', data=data, ax=axes[1,1])
axes[1,1].set_title('Survived')
plt.subplots_adjust(wspace=0.2,hspace=0.5)
plt.show()

majority of the passenger from either class embarked from S

In [ ]:
data['Embarked'].fillna("S", inplace=True)
data["Embarked"].isnull().any()

In [ ]:
pd.crosstab(data['SibSp'],data['Survived'],margins=True).T

In [ ]:
sns.barplot(x='Parch',y='Survived',data=data)
plt.title('Parch vs Survived')
plt.show()

# Feature Engineering 

In [ ]:
data.head()

In [ ]:
data['Family_size'] = 0
data['Family_size'] = data['Parch'] + data['SibSp']
data['Alone'] = 0
data.loc[data['Family_size']==0, 'Alone'] = 1

In [ ]:
pd.crosstab(data['Family_size'],data['Survived'], margins=True).T

### converting continous to categroical 

Age is a continous feature, there is a problem with Continous Variables in Machine Learning Models.

Eg:If I say to group or arrange Sports Person by Sex, We can easily segregate them by Male and Female.

Now if I say to group them by their Age, then how would you do it? If there are 30 Persons, there may be 30 age values. Now this is problematic.

We need to convert these continous values into categorical values by either Binning or Normalisation. I will be using binning i.e group a range of ages into a single bin or assign them a single value

In [ ]:
bands = max(data['Age'])/5
print(bands)

In [ ]:
data['Age_band'] = 0
data.loc[data['Age'] <= 16, 'Age_band'] = 0
data.loc[(data['Age']>16) & (data['Age']<=32), 'Age_band'] = 1
data.loc[(data['Age']>32) & (data['Age']<=48), 'Age_band'] = 2
data.loc[(data['Age']>48) & (data['Age']<=64), 'Age_band'] = 3
data.loc[(data['Age']>64), 'Age_band'] = 4
data.head(3)

### Converting String into numeric

In [ ]:
data['Sex'].replace(['male', 'female'], [0,1], inplace=True)
data['Embarked'].replace(['S','C','Q'], [0,1,2], inplace=True)
data['Initial'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4], inplace=True)
data.head(3)

In [ ]:
feature = ['PassengerId','Survived','Name', 'Age','Ticket','Fare', 'Cabin']
X = data.drop(feature, axis=1)
y = data['Survived']
y.dropna()

# Predictive Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=47) # For feature data

## Logistic Regression

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
lr_y_pred = log_reg.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_y_pred)



print(f'The Accuracy of the Logistic Regression is : {round(lr_accuracy,2)}')


## Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_y_pred)


print(f'The Accuracy of the Decision Tree is : {round(dt_accuracy,2)}')

## KNN

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_y_pred = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_y_pred)


print(f'The Accuracy of the KNN is : {round(knn_accuracy,2)}')


## Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)

print(f"The Accuracy of Random Forest: {round(rf_accuracy,2)}")

# Cross Validation 

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict

models = {"Logistic Regression":LogisticRegression(),
          "Decision Tree":DecisionTreeClassifier(),
          "KNN":KNeighborsClassifier(), 
          "Random Forest":RandomForestClassifier()}

results = []


for model in models.values():
    kf = KFold(n_splits=10, random_state=47, shuffle=True)
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf)
    results.append(cv_scores)

plt.boxplot(results, labels=models.keys())
plt.show()

## Confusion Matrix 
Confusion Matrix gives the number of correct and incorrect classifications made by the classifier.

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(12,6))
log_pred = cross_val_predict(LogisticRegression(), X,y, cv=10)
sns.heatmap(confusion_matrix(y, log_pred), annot=True, ax=axes[0,0],fmt='2.0f')
axes[0,0].set_title("Matrix for Logistic Regression")

dt_pred = cross_val_predict(DecisionTreeClassifier(), X,y, cv=10)
sns.heatmap(confusion_matrix(y, dt_pred), annot=True, ax=axes[0,1],fmt='2.0f')
axes[0,1].set_title("Matrix for Decision Tree")

knn_pred = cross_val_predict(KNeighborsClassifier(), X,y, cv=10)
sns.heatmap(confusion_matrix(y, knn_pred), annot=True, ax=axes[1,0],fmt='2.0f')
axes[1,0].set_title("Matrix for KNN")

rf_pred = cross_val_predict(RandomForestClassifier(), X,y, cv=10)
sns.heatmap(confusion_matrix(y, rf_pred), annot=True, ax=axes[1,1],fmt='2.0f')
axes[1,1].set_title("Matrix for Random Forest")

plt.subplots_adjust(hspace=.2,wspace=.2)
plt.show()

# Hyper Tunning Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV

n_estimators = range(100,1000,100)

params = {'n_estimators':n_estimators,
          'max_depth': [None, 5, 10],}

gd_rf = GridSearchCV(rf, params, verbose=True)
gd_rf.fit(X_train, y_train)

print("Tuned Random Forest Parameters: {}".format(gd_rf.best_params_))
print("Tuned Random Forest Best Accuracy Score: {}".format(gd_rf.best_score_))

# Ensembling

Ensembling is a good way to increase the accuracy or performance of a model. In simple words, it is the combination of various simple models to create a single powerful model.

Lets say we want to buy a phone and ask many people about it based on various parameters. So then we can make a strong judgement about a single product after analysing all different parameters. This is Ensembling, which improves the stability of the model. Ensembling can be done in ways like:

1)Voting Classifier

2)Bagging

3)Boosting

## Voting Classifier

It is the simplest way of combining predictions from many different simple machine learning models. It gives an average prediction result based on the prediction of all the submodels. The submodels or the basemodels are all of diiferent type

In [ ]:
from sklearn.ensemble import VotingClassifier

classifiers = [('Logistic Regression',log_reg),('KNN',knn),('Decision Tree',dt),('Random Forest', rf)]


for name, model in  classifiers:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name}: {accuracy_score(y_test, y_pred)*100}")

print("="*100)

vc = VotingClassifier(classifiers, voting='soft')

vc.fit(X_train, y_train)

print('The accuracy for ensembled model is:',vc.score(X_test,y_test))
cross=cross_val_score(vc ,X,y, cv = 10,scoring = "accuracy")
print('The cross validated score is',cross.mean())

## Bagging

Bagging is a general ensemble method. It works by applying similar classifiers on small partitions of the dataset and then taking the average of all the predictions. Due to the averaging,there is reduction in variance. Unlike Voting Classifier, Bagging makes use of similar classifiers.

In [ ]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(dt, random_state=47, n_estimators=100)
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)

print('The accuracy for bagged Decision Tree is:',accuracy_score(y_pred, y_test))
result=cross_val_score(bag,X,y,cv=10,scoring='accuracy')
print('The cross validated score for bagged Decision Tree is:',result.mean())

## Boosting

Boosting is an ensembling technique which uses sequential learning of classifiers. It is a step by step enhancement of a weak model.Boosting works as follows:

A model is first trained on the complete dataset. Now the model will get some instances right while some wrong. Now in the next iteration, the learner will focus more on the wrongly predicted instances or give more weight to it. Thus it will try to predict the wrong instance correctly. Now this iterative process continous, and new classifers are added to the model until the limit is reached on the accuracy.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)

result = cross_val_score(ada, X, y, cv=10, scoring='accuracy')

print('The cross validated score for AdaBoost is:',result.mean())

In [ ]:
n_estimators=list(range(100,1100,100))

learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]

hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}

gd = GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True)

gd.fit(X,y)

print(gd.best_score_)
print(gd.best_estimator_)

In [ ]:
# tunned Ada boost classifier 
ada_tuned = AdaBoostClassifier(n_estimators=200, learning_rate=0.05, random_state=47)
ada_result = cross_val_predict(ada_tuned, X,y, cv=10)
# tunned Random Forest 
rf_tuned = RandomForestClassifier(n_estimators=200, max_depth=5)
rf_result = cross_val_predict(rf_tuned, X,y, cv=10)

fig , axes = plt.subplots(1,2,figsize=(12,6))
sns.heatmap(confusion_matrix(y, rf_result), annot=True, fmt='2.0f', ax=axes[0])
axes[0].set_title("Random Forest Tunned")
sns.heatmap(confusion_matrix(y, ada_result), annot=True, fmt='2.0f', ax=axes[1])
axes[1].set_title("AdaBoost Tunned")

plt.show()

### Important Features

In [ ]:
rf_tuned.fit(X_train, y_train)
rf_importances = pd.Series(rf_tuned.feature_importances_, X_train.columns).sort_values(ascending=True)

ada_tuned.fit(X_train, y_train)
ada_importances = pd.Series(ada_tuned.feature_importances_, X_train.columns).sort_values(ascending=True)


In [ ]:
fig, axes= plt.subplots(1,2, figsize=(12,6))
# Plot Random Forest feature importances
axes[0].barh(range(len(rf_importances)), rf_importances, align='center')
axes[0].set_yticks(range(len(rf_importances)))
axes[0].set_yticklabels(X.columns)  # Assuming X is a DataFrame
axes[0].set_title("Random Forest Feature Importances")

# Plot AdaBoost feature importances
axes[1].barh(range(len(ada_importances)), ada_importances, align='center')
axes[1].set_yticks(range(len(ada_importances)))
axes[1].set_yticklabels(X.columns)  # Assuming X is a DataFrame
axes[1].set_title("AdaBoost Feature Importances")

plt.tight_layout()
plt.show()

In [ ]:
predictions = ada_tuned.predict(X_test)


aligned_test_data = test_data.iloc[:len(predictions)]
output = pd.DataFrame({'PassengerId':  aligned_test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Submission was successfully saved!")

In [ ]:
output.value_counts('Survived')